In [ ]:
import requests
url='https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip'

In [ ]:
#pull the zip file to computer
s1=requests.get(url)

In [ ]:
#make a staging dictionary
import os
staging_d='staging'
os.mkdir(staging_d)

In [ ]:
os.path.isdir(staging_d)

In [ ]:
zipN=os.path.join(staging_d,'Hospital_Revised_Flatfiles')

In [ ]:
zipN

In [ ]:
zf=open(zipN,'wb')

In [ ]:
zf.write(s1.content)

In [ ]:
zf.close

In [ ]:
import zipfile

In [ ]:
#read zip file
zr=zipfile.ZipFile(zipN,'r')

In [ ]:
#extract all members from the archive to the current working directory
zr.extractall(staging_d)

In [ ]:
zr.close()

In [ ]:
import sqlite3

In [ ]:
#create a databas
mc=sqlite3.connect('medicare_hospital_compare.db')

In [ ]:
#make a database cursor
c=mc.cursor()
c

In [ ]:
sql_str='drop table if exists my_table'

In [ ]:
c.execute(sql_str)

In [ ]:
import csv
import pandas as pd
from pandas import Series,DataFrame
import glob

In [ ]:
path=os.getcwd()
path

In [ ]:
newpath=os.path.join(path,'staging')
newpath


In [ ]:
#extract all csv files 
files1=glob.glob(os.path.join("staging" + "/*.csv"))

In [ ]:
#read in cp1252, output csv in utf-8
dict_={}
for file in files1:
    dict_[file]=pd.read_csv(file, header=0, dtype=str, encoding='cp1252')
for file in dict_:
    dict_[file].to_csv(file, encoding='utf-8',index=0)

In [ ]:
import string
l=list(string.ascii_lowercase)+list(string.ascii_uppercase)

In [ ]:
#working with the column name as required 
for file in dict_:
    dict_[file]=pd.read_csv(file, header=0, dtype=str)
    for col in dict_[file]:
        dict_[file].columns=map(str.lower,dict_[file].columns)
        dict_[file].columns=[col.strip().replace(" ", '_').replace('-', '_').replace('%', 'pct').replace('/', '_') for col in dict_[file].columns]
    for col in dict_[file]:
        if col[0] in l:
            pass
        else:
            dict_[file].rename(columns={col:'c'+col},inplace=True)

Index(['provider_id', 'vha_facility', 'address', 'city', 'state', 'zip_code',
       'measureid', 'technical_measure_title',
       'measure_as_posted_on_hospital_compare', 'observed_rate',
       'expected_rate', 'risk_adjusted_rate', '95pct_confidence_lower_limit',
       '95pct_confidence_lower_upper_limit', 'footnotes', 'date_range'],
      dtype='object')

In [ ]:
#rewrite csv files with transformed column name
for file in dict_:
    dict_[file].to_csv(file, encoding='utf-8',index=0)

In [ ]:
#transform file name
for file in os.listdir("staging"):
    if file[-3:]=='csv':
        os.rename("staging"+'\\'+file, "staging"+'\\'+file.lower())
        if file[0] in l:
            pass
        else:
            file=file.replace(file,'t'+file)

In [ ]:
#transform file name
for file in os.listdir("staging"):
    if file[-3:]=='csv':
        os.rename("staging"+'\\'+file,"staging"+'\\'+file.replace(' ','_').replace('-','_').replace('%','pct').replace('/','_'))
        

In [ ]:
#list all csv file name
csvn=[]
for file in os.listdir('staging'):
    if file[-3:]=='csv':
        csvn.append(file)

In [ ]:
#see if file numbers match
len([name for name in files1])==len([name for name in csvn])

In [ ]:
#import csv files as database tables
for i in range(62):
    dict_[files1[i]].to_sql(csvn[i][:-4], mc, if_exists='append', index=False)
    

In [ ]:
#drop table with corrupt data
c.execute('DROP TABLE fy2015_percent_change_in_medicare_payments')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
url2='http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx'

In [ ]:
#pull xlsx file to computer
r2=requests.get(url2)

In [ ]:
f=open("hospital_ranking_focus_states.xlsx","wb")

In [ ]:
f.write(r2.content)

In [ ]:
#save the file
f.close()

In [ ]:
import openpyxl
book=openpyxl.load_workbook('hospital_ranking_focus_states.xlsx')

In [ ]:
#geet sheet name in the file
for sheet_name in book.get_sheet_names():
    print(sheet_name)

In [ ]:
sheet_a_name = book.get_sheet_names()[0]
sheet_a = book.get_sheet_by_name(sheet_a_name)

In [ ]:
sheet_b_name = book.get_sheet_names()[1]
sheet_b = book.get_sheet_by_name(sheet_b_name)

In [ ]:
#print sheet a
i = 1
while sheet_a.cell(row = i, column = 1).value != None:
    print(sheet_a.cell(row = i,column = 1).value, "|", sheet_a.cell(row = i, column = 2).value)
    i += 1

In [ ]:
#print sheet b
i = 1
while sheet_b.cell(row = i, column = 1).value != None:
    print(sheet_b.cell(row = i,column = 1).value, "|", sheet_b.cell(row = i, column = 2).value)
    i += 1

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [ ]:
#get the provider id from sheet a and make a list
topP=[]
i = 2
while sheet_a.cell(row = i, column = 1).value != None:
    topP.append(sheet_a.cell(row = i,column = 1).value)
    i+=1

In [ ]:
#get top 100 provider id and make a df
Rank=pd.DataFrame()
Rank['Provider ID']=topP[:100]


In [ ]:
#query data out from database and put to df, rename columns
NW=pd.read_sql('SELECT provider_id, hospital_name, city, state, county_name from hospital_general_information',mc)
NW=NW.rename(columns = {'provider_id':'Provider ID'}).rename(columns={'city':'City'}).rename(columns={'hospital_name':'Hospital Name'}).rename(columns = {'state':'State'}).rename(columns = {'county_name':'County'})


In [ ]:
#merge data from two df where provider id matches
Nationwide=Rank.merge(NW,how='inner',on=['Provider ID'])

In [ ]:
#get all provider id and make a df
ProviderRank=pd.DataFrame()
ProviderRank['Provider ID']=topP


In [ ]:
#merge data from two df where provider id matches
match=ProviderRank.merge(NW,how='inner',on=['Provider ID'])


In [ ]:
#get top 100 provider info in each state 
TX=match.loc[match['State']=='TX']
Texas=TX[:100]

CA=match.loc[match['State']=='CA']
California=CA[:100]

FL=match.loc[match['State']=='FL']
Florida=FL[:100]

GA=match.loc[match['State']=='GA']
Georgia=GA[:100]

IL=match.loc[match['State']=='IL']
Illinois=IL[:100]

KS=match.loc[match['State']=='KS']
Kansas=KS[:100]

MI=match.loc[match['State']=='MI']
Michigan=MI[:100]

NY=match.loc[match['State']=='NY']
New_York=NY[:100]

OH=match.loc[match['State']=='OH']
Ohio=OH[:100]

PA=match.loc[match['State']=='PA']
Pennsylvania=PA[:100]

#Pennsylvania.index=range(100)

In [ ]:
#write df to xlsx
hr=pd.ExcelWriter('hospital_ranking.xlsx')

Nationwide.to_excel(hr,'Nationwide',index=False)
California.to_excel(hr,'California',index=False)
Florida.to_excel(hr,'Florida',index=False)
Georgia.to_excel(hr,'Georgia',index=False)
Illinois.to_excel(hr,'Illinois',index=False)
Kansas.to_excel(hr,'Kansas',index=False)
Michigan.to_excel(hr,'Michigan',index=False)
New_York.to_excel(hr,'New_York',index=False)
Ohio.to_excel(hr,'Ohio',index=False)
Pennsylvania.to_excel(hr,'Pennsylvania',index=False)
Texas.to_excel(hr,'Texas',index=False)

hr.save()

In [ ]:
#query data out to df
STAT=pd.read_sql('SELECT state, measure_id, measure_name, score From timely_and_effective_care___hospital',mc)

In [ ]:
#get unique measure id and measure name, make a list with state name
measureID=STAT.measure_id.unique()
measureID
measureName=STAT.measure_name.unique()
measureName
stateName=[]
i = 2
while sheet_b.cell(row = i,column=2).value!=None:
    stateName.append(sheet_b.cell(row=i,column=2).value)
    i+=1
stateName

In [ ]:
#calculate the minimum, maximum, average, and standard deviation for each measure for all hospitals nationwide
df=pd.DataFrame(columns=['Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation'])

for i in range(22):
    a=STAT.loc[STAT['measure_id']==measureID[i]] 
    b=(a.drop('score', axis=1).join(a['score'].apply(pd.to_numeric, errors='coerce')))#with option 'coerce', such that all non-numeric entries are converted to NaN
    c=b[b['score'].notnull()]
    if c['score'].isnull().values.all():
        #can add 'print('There is no record for this measure: '+measureID[i])'
        pass
    else:
        mini=min(c['score'])
        maximum=max(c['score'])
        average=c['score'].mean()
        std=c['score'].std()
        df.loc[i]=[measureID[i],measureName[i],mini,maximum,average,std]
       

In [ ]:
nationwild=df
nationwild

In [ ]:
#a function to get the minimum, maximum, average and standard deviation for each measure for all hospitals in a specific state in stateName


def MS(j):
    dfstate=pd.DataFrame(columns=['Measure ID','Measure Name','Minimum','Maximum','Average','Standard Deviation'])
    if j in range(10):
        df2=STAT.loc[STAT['state']==stateName[j]]#use index to specify the state
        for i in range(22):
            df3=df2.loc[STAT['measure_id']==measureID[i]]
            df4=(df3.drop('score', axis=1).join(df3['score'].apply(pd.to_numeric, errors='coerce')))
            df5=df4[df4['score'].notnull()]
            if df5['score'].isnull().values.all():
                #can add 'print('There is no record in '+stateName[j]+ ' for measure: '+measureID[i])'
                pass
            else:
                mini=min(df5['score'])
                maximum=max(df5['score'])
                average=df5['score'].mean()
                std=df5['score'].std()
                dfstate.loc[i]=[measureID[i],measureName[i],mini,maximum,average,std]
        return dfstate

In [ ]:
CA=MS(0)
FL=MS(1)
GA=MS(2)
IL=MS(3)
KS=MS(4)
MI=MS(5)
NY=MS(6)
OH=MS(7)
PA=MS(8)
TX=MS(9)

In [ ]:
#write df to xlsx file
ms=pd.ExcelWriter('measure_statistics.xlsx')

nationwild.to_excel(ms,'Nationwide',index=False)
CA.to_excel(ms,'California',index=False)
FL.to_excel(ms,'Florida',index=False)
GA.to_excel(ms,'Georgia',index=False)
IL.to_excel(ms,'Illinois',index=False)
KS.to_excel(ms,'Kansas',index=False)
MI.to_excel(ms,'Michigan',index=False)
NY.to_excel(ms,'New_York',index=False)
OH.to_excel(ms,'Ohio',index=False)
PA.to_excel(ms,'Pennsylvania',index=False)
TX.to_excel(ms,'Texas',index=False)

ms.save()